In [1]:
import cv2
import numpy as np
from skimage import exposure

In [2]:
def apply_gamma_correction(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** invGamma * 255 for i in np.arange(256)]).astype("uint8")
    return cv2.LUT(image, table)

In [3]:
def contrast_stretching(image):
    p2, p98 = np.percentile(image, (2, 98))
    return exposure.rescale_intensity(image, in_range=(p2, p98))

In [4]:
def get_dark_channel(image, window_size=15):
    min_channel = np.min(image, axis=2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (window_size, window_size))
    dark_channel = cv2.erode(min_channel, kernel)
    return dark_channel

In [5]:

def estimate_atmospheric_light(image, dark_channel, top_percent=0.001):
    flat_image = image.reshape((-1, 3))
    flat_dark = dark_channel.ravel()

    num_pixels = dark_channel.size
    num_brightest = int(max(num_pixels * top_percent, 1))

    indices = np.argpartition(-flat_dark, num_brightest)[:num_brightest]
    brightest = flat_image[indices]

    atmospheric_light = np.mean(brightest, axis=0)
    return atmospheric_light


In [6]:
def estimate_transmission(image, atmospheric_light, omega=0.95, window_size=15):
    norm_image = image / atmospheric_light
    dark_channel = get_dark_channel(norm_image, window_size)
    transmission = 1 - omega * dark_channel
    return transmission

In [7]:
def recover_image(image, transmission, atmospheric_light, t0=0.1):
    transmission = np.maximum(transmission, t0)
    result = (image - atmospheric_light) / transmission[:, :, np.newaxis] + atmospheric_light
    result = np.clip(result, 0, 255).astype(np.uint8)
    return result

In [8]:
def dehaze(image):
    dark_channel = get_dark_channel(image)
    atmospheric_light = estimate_atmospheric_light(image, dark_channel)
    transmission = estimate_transmission(image, atmospheric_light)
    result = recover_image(image, transmission, atmospheric_light)
    return result

In [9]:
def calculate_snr(image):
    mean_signal = np.mean(image)
    std_noise = np.std(image)
    if std_noise == 0:  
        return float('inf') 
    return mean_signal / std_noise

In [10]:
def reduce_noise(image, kernel_size=(5, 5), sigma=0):
    return cv2.GaussianBlur(image, kernel_size, sigma)

In [11]:
def sharpen_image(image, sigma=1.0, strength=1.5):
    blurred = cv2.GaussianBlur(image, (0, 0), sigma)
    sharpened = cv2.addWeighted(image, 1 + strength, blurred, -strength, 0)
    return sharpened

In [12]:
def adaptive_restoration(image):
    
    brightness = np.mean(image)
    if brightness < 50:  
        print("Applying gamma correction")
        image = apply_gamma_correction(image, gamma=1.5)

    
    if np.std(image) < 50:  
        print("Applying contrast stretching")
        image = contrast_stretching(image)

    if np.mean(image) < 128: 
        print("Applying dehazing")
        image = dehaze(image)

    snr = calculate_snr(image)
    if snr < 4.0:  
        print(f"Applying noise reduction as SNR = {snr:.2f}")
        image = reduce_noise(image)

    sharpened_image = sharpen_image(image)
    print("Image sharpening applied")

    return sharpened_image

In [14]:
image = cv2.imread('images\\3.jpg', cv2.IMREAD_COLOR)  
restored_image = adaptive_restoration(image)

cv2.imwrite("results\RestoredImage3.jpg", restored_image)

<>:4: SyntaxWarning: invalid escape sequence '\R'
<>:4: SyntaxWarning: invalid escape sequence '\R'
C:\Users\ekru\AppData\Local\Temp\ipykernel_22536\1667527757.py:4: SyntaxWarning: invalid escape sequence '\R'
  cv2.imwrite("results\RestoredImage3.jpg", restored_image)


Applying contrast stretching
Applying dehazing
Applying noise reduction as SNR = 1.85
Image sharpening applied


True

In [152]:
image = cv2.imread('images\old_photo_07.jpg', cv2.IMREAD_COLOR) 
restored_image = adaptive_restoration(image)

cv2.imwrite("results\RestoredImage7.jpg", restored_image)

Applying contrast stretching
Applying noise reduction as SNR = 2.53
Image sharpening applied


True

In [153]:
image = cv2.imread('images\old_photo_09.jpeg', cv2.IMREAD_COLOR) 
restored_image = adaptive_restoration(image)

cv2.imwrite("results\RestoredImage9.jpg", restored_image)

Applying dehazing
Applying noise reduction as SNR = 1.86
Image sharpening applied


True

In [154]:
image = cv2.imread('images\\11.jpg', cv2.IMREAD_COLOR)  
restored_image = adaptive_restoration(image)

cv2.imwrite("results\RestoredImage11.jpg", restored_image)

Applying gamma correction
Applying contrast stretching
Applying dehazing
Applying noise reduction as SNR = 1.26
Image sharpening applied


True

In [155]:
image = cv2.imread('images\\26.jpg', cv2.IMREAD_COLOR) 
restored_image = adaptive_restoration(image)

cv2.imwrite("results\RestoredImage26.jpg", restored_image)

Applying contrast stretching
Applying dehazing
Applying noise reduction as SNR = 1.23
Image sharpening applied


True

In [156]:
image = cv2.imread('images\\14.jpg', cv2.IMREAD_COLOR)  
restored_image = adaptive_restoration(image)

cv2.imwrite("results\RestoredImage14.jpg", restored_image)

Applying dehazing
Applying noise reduction as SNR = 1.23
Image sharpening applied


True

In [158]:
image = cv2.imread('images\\21.jpg', cv2.IMREAD_COLOR)  
restored_image = adaptive_restoration(image)

cv2.imwrite("results\RestoredImage-test.jpg", restored_image)

Applying gamma correction
Applying contrast stretching
Applying noise reduction as SNR = 2.65
Image sharpening applied


True